# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 27 2022 3:01PM,238312,10,0085720575,ISDIN Corporation,Released
1,May 27 2022 3:01PM,238312,10,0085720580,ISDIN Corporation,Released
2,May 27 2022 3:01PM,238312,10,0085720582,ISDIN Corporation,Released
3,May 27 2022 3:01PM,238312,10,0085720583,ISDIN Corporation,Released
4,May 27 2022 3:01PM,238312,10,0085720588,ISDIN Corporation,Released
5,May 27 2022 3:01PM,238312,10,0085720577,ISDIN Corporation,Released
6,May 27 2022 3:01PM,238312,10,0085720618,ISDIN Corporation,Released
7,May 27 2022 3:01PM,238312,10,0085720619,ISDIN Corporation,Released
8,May 27 2022 3:01PM,238312,10,0085720622,ISDIN Corporation,Released
9,May 27 2022 3:01PM,238312,10,0085720624,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,238309,Released,34
29,238310,Released,3
30,238311,Released,3
31,238312,Released,28
32,238313,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238309,NaN,NaN,34.0
238310,NaN,NaN,3.0
238311,NaN,NaN,3.0
238312,NaN,NaN,28.0
238313,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238212,0.0,1.0,1.0
238219,0.0,0.0,1.0
238225,0.0,8.0,1.0
238253,0.0,0.0,2.0
238255,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238212,0,1,1
238219,0,0,1
238225,0,8,1
238253,0,0,2
238255,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238212,0,1,1
1,238219,0,0,1
2,238225,0,8,1
3,238253,0,0,2
4,238255,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238212,,1,1
1,238219,,,1
2,238225,,8,1
3,238253,,,2
4,238255,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 27 2022 3:01PM,238312,10,ISDIN Corporation
28,May 27 2022 2:58PM,238311,10,ISDIN Corporation
31,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated"
32,May 27 2022 2:56PM,238310,10,"Nextsource Biotechnology, LLC"
35,May 27 2022 2:55PM,238309,10,ISDIN Corporation
69,May 27 2022 2:47PM,238305,10,ISDIN Corporation
95,May 27 2022 2:47PM,238301,10,ISDIN Corporation
124,May 27 2022 2:45PM,238306,10,ISDIN Corporation
142,May 27 2022 2:40PM,238304,16,Sartorius Bioprocess Solutions
143,May 27 2022 2:39PM,238303,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 27 2022 3:01PM,238312,10,ISDIN Corporation,,,28
1,May 27 2022 2:58PM,238311,10,ISDIN Corporation,,,3
2,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",,,1
3,May 27 2022 2:56PM,238310,10,"Nextsource Biotechnology, LLC",,,3
4,May 27 2022 2:55PM,238309,10,ISDIN Corporation,,,34
5,May 27 2022 2:47PM,238305,10,ISDIN Corporation,,,26
6,May 27 2022 2:47PM,238301,10,ISDIN Corporation,,,29
7,May 27 2022 2:45PM,238306,10,ISDIN Corporation,,,18
8,May 27 2022 2:40PM,238304,16,Sartorius Bioprocess Solutions,,,1
9,May 27 2022 2:39PM,238303,16,Sartorius Bioprocess Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 3:01PM,238312,10,ISDIN Corporation,28,,
1,May 27 2022 2:58PM,238311,10,ISDIN Corporation,3,,
2,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",1,,
3,May 27 2022 2:56PM,238310,10,"Nextsource Biotechnology, LLC",3,,
4,May 27 2022 2:55PM,238309,10,ISDIN Corporation,34,,
5,May 27 2022 2:47PM,238305,10,ISDIN Corporation,26,,
6,May 27 2022 2:47PM,238301,10,ISDIN Corporation,29,,
7,May 27 2022 2:45PM,238306,10,ISDIN Corporation,18,,
8,May 27 2022 2:40PM,238304,16,Sartorius Bioprocess Solutions,1,,
9,May 27 2022 2:39PM,238303,16,Sartorius Bioprocess Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 3:01PM,238312,10,ISDIN Corporation,28,,
1,May 27 2022 2:58PM,238311,10,ISDIN Corporation,3,,
2,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",1,,
3,May 27 2022 2:56PM,238310,10,"Nextsource Biotechnology, LLC",3,,
4,May 27 2022 2:55PM,238309,10,ISDIN Corporation,34,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 3:01PM,238312,10,ISDIN Corporation,28.0,NaN,NaN
1,May 27 2022 2:58PM,238311,10,ISDIN Corporation,3.0,NaN,NaN
2,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",1.0,NaN,NaN
3,May 27 2022 2:56PM,238310,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
4,May 27 2022 2:55PM,238309,10,ISDIN Corporation,34.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 3:01PM,238312,10,ISDIN Corporation,28.0,0.0,0.0
1,May 27 2022 2:58PM,238311,10,ISDIN Corporation,3.0,0.0,0.0
2,May 27 2022 2:58PM,238313,102,"Senseonics, Incorporated",1.0,0.0,0.0
3,May 27 2022 2:56PM,238310,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
4,May 27 2022 2:55PM,238309,10,ISDIN Corporation,34.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3097757,154.0,23.0,4.0
102,238313,1.0,0.0,0.0
15,238279,1.0,4.0,0.0
16,1429632,5.0,9.0,0.0
17,238256,1.0,0.0,0.0
19,476597,2.0,0.0,0.0
20,476531,15.0,19.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3097757,154.0,23.0,4.0
1,102,238313,1.0,0.0,0.0
2,15,238279,1.0,4.0,0.0
3,16,1429632,5.0,9.0,0.0
4,17,238256,1.0,0.0,0.0
5,19,476597,2.0,0.0,0.0
6,20,476531,15.0,19.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,154.0,23.0,4.0
1,102,1.0,0.0,0.0
2,15,1.0,4.0,0.0
3,16,5.0,9.0,0.0
4,17,1.0,0.0,0.0
5,19,2.0,0.0,0.0
6,20,15.0,19.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,154.0
1,102,Released,1.0
2,15,Released,1.0
3,16,Released,5.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,17,19,20
Status,,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,23.0,0.0,4.0,9.0,0.0,0.0,19.0
Released,154.0,1.0,1.0,5.0,1.0,2.0,15.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,17,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,0.0,4.0,9.0,0.0,0.0,19.0
2,Released,154.0,1.0,1.0,5.0,1.0,2.0,15.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,17,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,23.0,0.0,4.0,9.0,0.0,0.0,19.0
2,Released,154.0,1.0,1.0,5.0,1.0,2.0,15.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()